## 법인 등록번호 가져오기
## ==============
### tool
### 1. selenium
#### - pip install selenium
### 2. Chrome driver (직접 다운로드)
## ==============
### 크롬 정보 필요 -> chrome://version
###	91.0.4472.114

In [2]:
from selenium import webdriver
# 페이지 불러올 때 까지 기다기기 위함
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

corp = "삼성전자"
browser = webdriver.Chrome('./chromedriver.exe')
# 주소 get
browser.get("http://dart.fss.or.kr/")

# 검색창 element
searchElem = browser.find_element_by_id("textCrpNm")

# 검색창 click
# searchElem.click()

# 키 값 입력을 위함
from selenium.webdriver.common.keys import Keys

# 검색
searchElem.send_keys(corp) # 여기선 위의 import 필요 없음.
searchElem.send_keys(Keys.ENTER)

# 기업 리스트 조회까지 대기
try:
    # 기업 링크 클릭
    corpLink = WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.LINK_TEXT, corp)))
    corpLink[0].click()
finally:
    print("공시 System....")

# 법인등록번호 Xpath
xpath = "/html/body/div[8]/div[2]/div[1]/div/div/div/div/div/div/div/div[2]/div/table/tbody/tr[7]/td"
CorpNum = ""
# 기업 정보 팝업이 될 때 까지 대기
try:
    CorpNumber = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    CorpNum = CorpNumber.text
finally: 
    print("정보 조회...")
    browser.quit()

# 사용할 법인 등록 번호
CorpNum = CorpNum[:6]+CorpNum[7:]
print(CorpNum)


공시 System....
정보 조회...
1301110006246


# 기업의 손익계산서, 재무상태표 정보 출력 및 활용
### Ex) 삼성전자
### 법인등록번호 130111-0006246
### 사업자등록번호 124-81-00998
### 회계연도 2020

# ------------------------------
### tool: requests
#### - python -m pip install requests

### tool: xmltodict
#### - pip install xmltodict
# ------------------------------
### 참고 : https://www.geeksforgeeks.org/python-xml-to-json/ (XML -> JSON 변환법)


In [3]:
#영미식 단위로 끊어주는 함수

def money(mon):
    a = mon
    result = str(mon)
    finalresult = ""
    count = 0
    while (a > 0):
        a = a//1000
        if(a != 0):
            count+=1
    for i in range(count):
        info = -3*count
        index = info+i*3
        result = result[:index] + ',' + result[index:]
        
    return result
#----------------
#test
money(1320000)
#result = 1,320,000
#----------------
# O(n)

'1,320,000'

In [4]:
import requests as rq
import json, xmltodict
#서비스 키는 apiKey.py를 통해 관리
import apiKey

ServiceKey = apiKey.serviceKey

Year = "2020"

#법인등록번호는 위에서 스크래핑 해온 데이터 "CorpNum"사용.
#손익계산서 URL
S_I = "http://apis.data.go.kr/1160100/service/GetFinaStatInfoService/getIncoStat?serviceKey="+ServiceKey+"&numOfRows=10&pageNo=1&numOfRows=xml&crno="+CorpNum+"&bizYear="+Year
#재무상태표 URL
S_FP = "http://apis.data.go.kr/1160100/service/GetFinaStatInfoService/getBs?serviceKey="+ServiceKey+"&numOfRows=18&pageNo=1&numOfRows=xml&crno="+CorpNum+"&bizYear="+Year

### 손익계산서 처리

In [5]:
S_I_content = rq.get(S_I).content
S_I_dict = xmltodict.parse(S_I_content)

In [6]:
#string 형태의 json
S_I_jsonString = json.dumps(S_I_dict['response']['body']['items']['item'], ensure_ascii=False)

In [7]:
#string -> json
S_I_jsonObj = json.loads(S_I_jsonString)
print(S_I_jsonObj)

[{'acitId': 'dart_OperatingIncomeLoss', 'acitNm': '영업이익', 'basDt': '20201231', 'bizYear': '2020', 'bpvtrAcitAmt': '58886669000000', 'crno': '1301110006246', 'crtmAcitAmt': '35993876000000', 'fnclDcd': 'PL_ifrs-full_ConsolidatedMember', 'fnclDcdNm': '연결재무제표 [member]', 'lsqtAcitAmt': '0', 'pvtrAcitAmt': '27768509000000', 'thqrAcitAmt': '0'}, {'acitId': 'ifrs-full_ProfitLoss', 'acitNm': '당기순이익(손실)', 'basDt': '20201231', 'bizYear': '2020', 'bpvtrAcitAmt': '44344857000000', 'crno': '1301110006246', 'crtmAcitAmt': '26407832000000', 'fnclDcd': 'PL_ifrs-full_ConsolidatedMember', 'fnclDcdNm': '연결재무제표 [member]', 'lsqtAcitAmt': '0', 'pvtrAcitAmt': '21738865000000', 'thqrAcitAmt': '0'}, {'acitId': 'ifrs-full_ProfitLossBeforeTax', 'acitNm': '법인세비용차감전순이익(손실)', 'basDt': '20201231', 'bizYear': '2020', 'bpvtrAcitAmt': '61159958000000', 'crno': '1301110006246', 'crtmAcitAmt': '36345117000000', 'fnclDcd': 'PL_ifrs-full_ConsolidatedMember', 'fnclDcdNm': '연결재무제표 [member]', 'lsqtAcitAmt': '0', 'pvtrAcitAmt'

In [8]:
for i in S_I_jsonObj:
    print('====',i['acitNm'],'====')
    print("당분기:", money(int(i['thqrAcitAmt']))+"원")
    print("당기:",money(int(i['crtmAcitAmt']))+"원")
    print("전분기:",money(int(i['lsqtAcitAmt']))+"원")
    print("전기:",money(int(i['pvtrAcitAmt']))+"원")
    print("전전기:",money(int(i['bpvtrAcitAmt']))+"원")
    if(i['fnclDcdNm'] == "연결재무제표 [member]"):
        print("재무제표 종류:","연결재무제표\n")
    elif i['fnclDcdNm'] == "별도재무제표 [member]":
        print("재무제표 종류:","별도재무제표\n")

==== 영업이익 ====
당분기: 0원
당기: 35,993,876,000,000원
전분기: 0원
전기: 27,768,509,000,000원
전전기: 58,886,669,000,000원
재무제표 종류: 연결재무제표

==== 당기순이익(손실) ====
당분기: 0원
당기: 26,407,832,000,000원
전분기: 0원
전기: 21,738,865,000,000원
전전기: 44,344,857,000,000원
재무제표 종류: 연결재무제표

==== 법인세비용차감전순이익(손실) ====
당분기: 0원
당기: 36,345,117,000,000원
전분기: 0원
전기: 30,432,189,000,000원
전전기: 61,159,958,000,000원
재무제표 종류: 연결재무제표

==== 수익(매출액) ====
당분기: 0원
당기: 236,806,988,000,000원
전분기: 0원
전기: 230,400,881,000,000원
전전기: 243,771,415,000,000원
재무제표 종류: 연결재무제표

==== 영업이익 ====
당분기: 0원
당기: 20,518,974,000,000원
전분기: 0원
전기: 14,115,067,000,000원
전전기: 43,699,451,000,000원
재무제표 종류: 별도재무제표

==== 당기순이익(손실) ====
당분기: 0원
당기: 15,615,018,000,000원
전분기: 0원
전기: 15,353,323,000,000원
전전기: 32,815,127,000,000원
재무제표 종류: 별도재무제표

==== 법인세비용차감전순이익(손실) ====
당분기: 0원
당기: 20,451,923,000,000원
전분기: 0원
전기: 19,032,469,000,000원
전전기: 44,398,855,000,000원
재무제표 종류: 별도재무제표

==== 수익(매출액) ====
당분기: 0원
당기: 166,311,191,000,000원
전분기: 0원
전기: 154,772,859,000,000원
전전기: 170,381,870,000,000원
재무제표 

### 재무상태표 처리

In [9]:
S_FP_content = rq.get(S_FP).content
S_FP_dict = xmltodict.parse(S_FP_content)

In [10]:
#string 형태의 json
S_FP_jsonString = json.dumps(S_FP_dict['response']['body']['items']['item'], ensure_ascii=False)

In [11]:
#string -> json
S_FP_jsonObj = json.loads(S_FP_jsonString)

In [12]:
for i in S_FP_jsonObj:
    print('====',i['acitNm'],'====')
    print("당분기:", money(int(i['thqrAcitAmt']))+"원")
    print("당기:",money(int(i['crtmAcitAmt']))+"원")
    print("전분기:",money(int(i['lsqtAcitAmt']))+"원")
    print("전기:",money(int(i['pvtrAcitAmt']))+"원")
    print("전전기:",money(int(i['bpvtrAcitAmt']))+"원")
    if(i['fnclDcdNm'] == "연결재무제표 [member]"):
        print("재무제표 종류:","연결재무제표\n")
    elif i['fnclDcdNm'] == "별도재무제표 [member]":
        print("재무제표 종류:","별도재무제표\n")

==== 자산총계 ====
당분기: 0원
당기: 378,235,718,000,000원
전분기: 0원
전기: 352,564,497,000,000원
전전기: 339,357,244,000,000원
재무제표 종류: 연결재무제표

==== 유동자산 ====
당분기: 0원
당기: 198,215,579,000,000원
전분기: 0원
전기: 181,385,260,000,000원
전전기: 174,697,424,000,000원
재무제표 종류: 연결재무제표

==== 유동부채 ====
당분기: 0원
당기: 75,604,351,000,000원
전분기: 0원
전기: 63,782,764,000,000원
전전기: 69,081,510,000,000원
재무제표 종류: 연결재무제표

==== 자본총계 ====
당분기: 0원
당기: 275,948,016,000,000원
전분기: 0원
전기: 262,880,421,000,000원
전전기: 247,753,177,000,000원
재무제표 종류: 연결재무제표

==== 자본금 ====
당분기: 0원
당기: 897,514,000,000원
전분기: 0원
전기: 897,514,000,000원
전전기: 897,514,000,000원
재무제표 종류: 연결재무제표

==== 부채총계 ====
당분기: 0원
당기: 102,287,702,000,000원
전분기: 0원
전기: 89,684,076,000,000원
전전기: 91,604,067,000,000원
재무제표 종류: 연결재무제표

==== 비유동자산 ====
당분기: 0원
당기: 180,020,139,000,000원
전분기: 0원
전기: 171,179,237,000,000원
전전기: 164,659,820,000,000원
재무제표 종류: 연결재무제표

==== 비유동부채 ====
당분기: 0원
당기: 26,683,351,000,000원
전분기: 0원
전기: 25,901,312,000,000원
전전기: 22,522,557,000,000원
재무제표 종류: 연결재무제표

==== 이익잉여금(결손금) ====
당분기: 0

In [13]:
''' 
@CopyRight 

2021-07-07
Republic of Korea

Park.Giyoun
Hankuk University of Foreign Studies
Industrial & Management Engineering
'''

# parameter L_asset's index
# ----손익계산서----
# 0 : 영업이익
# 1 : 당기순이익
# 2 : 법인세차감전순이익
# 3 : 수익 (매출)
# ----재무상태표---- 
# 4 : 자산총계
# 5 : 유동자산
# 6 : 유동부채
# 7 : 자본총계
# 8 : 자본금
# 9 : 부채총계
# 10: 비유동자산
# 11: 비유동부채
# 12: 이익잉여금 
# TOTAL : 13개의 인덱스

# result's index
# ----수익성비율----
# 0 : 총자산이익률(ROA)
# 1 : 매출액순이익률
# 2 : 매출총이익률
# 3 : 매출액영업이익률
# 4 : 매출액 계속사업이익률
# ----활동성비율----
# 5 : 총자산회전률
# 6 : 유동자산회전률
# 7 : 비유동자산회전률
# ----레버리지비율(부채비율)---- 
# 8 : 부채구성비율
# 9 : 부채비율 

def rateConverter (L_asset):
    result = []
    # 0 총자산이익률 ROA
    # = 매출액순이익률 * 총자산회전율 (당기순이익 / 총 자산)
    result.append(L_asset[1]/L_asset[4])

    # 1 매출액 순이익률 ROS
    # = 당기순이익 / 매출액
    result.append(L_asset[1]/L_asset[3])

    # 2 매출 총 이익률
    # = 매출 총 이익 / 매출액 [여기서 ROS와 매출 총 이익률은 같다.]
    result.append(L_asset[1]/L_asset[3])

    # 3 매출액 영업이익률
    # = 영업이익 / 매출액
    result.append(L_asset[0]/L_asset[3])

    # 4 매출액 계속사업이익률
    # = 계속사업이익(법인세 차감 전 이익) / 매출액
    result.append(L_asset[2]/L_asset[3])

    # 5 총자산회전률
    # = 매출액 / 총 자산
    result.append(L_asset[3]/L_asset[4])

    # 6 유동자산회전률
    # = 매출액 / 유동자산
    result.append(L_asset[3]/L_asset[5])

    # 7 비유동자산회전률
    # = 매출액 / 비유동자산
    result.append(L_asset[3]/L_asset[10])

    # 8 부채구성비율
    # = 총부채 / 총자산
    result.append(L_asset[9]/L_asset[4])
    
    # 9 부채비율
    # = 총부채 / 자기자본
    result.append(L_asset[9]/L_asset[7])

    return result

In [21]:
'''
손익계산서 정보, 재무상태표 정보 활용
리스트 [Assets] 만들기

 * 당기 정보만 활용

'''
Assets = []
#S_I_jsonObj
#S_FP_jsonObj

Assets.append(corp)
for i in range(4):
    Assets.append(int(S_I_jsonObj[i]['crtmAcitAmt'])/1000000)

for i in range(9):
    Assets.append(int(S_FP_jsonObj[i]['crtmAcitAmt'])/1000000)

print("Assets:",Assets)

Assets: ['삼성전자', 35993876.0, 26407832.0, 36345117.0, 236806988.0, 378235718.0, 198215579.0, 75604351.0, 275948016.0, 897514.0, 102287702.0, 180020139.0, 26683351.0, 271068211.0]


In [22]:
# 비율로 변환 후 출력 테스트
Rates = [corp]
Rates = Rates + rateConverter(Assets[1:])
print(Rates)

['삼성전자', 0.0698184511490266, 0.1115162699506148, 0.1115162699506148, 0.15199668009797077, 0.1534799175774323, 0.6260830924487147, 1.1946941264389719, 1.315447201160088, 0.27043374576274154, 0.3706774322305691]


In [23]:
import pandas as pd

_BasicData = [
    ("기업명", []),
    ("회계연도", []),
    ("매출액", []),
    ("영업이익", []),
    ("법인세차감전순이익", []),
    ("당기순이익", []),
    ("자산총계", []),
    ("유동자산", []),
    ("자본총계", []),
    ("자본금", []),
    ("부채총계", []),
    ("비유동부채", []),
    ("비유동자산", []),
    ("이익잉여금", [])
]
_RateData  = [
    ("기업명", []),
    ("ROA", []),
    ("ROS", []),
    ("매출총이익률", []),
    ("매출액영업이익률", []),
    ("매출액계속사업이익률", []),
    ("총자산회전율", []),
    ("유동자산회전율", []),
    ("비유동자산회전율", []),
    ("부채구성비율", []),
    ("부채비율", [])
]

for i in range(len(Assets)):
    _BasicData[i][1].append(Assets[i])
#========= 테스트용. (데이터프레임의 변수에는 각각 두개 이상의 요소들이 필요하다.)
for i in range(len(Assets)):
    _BasicData[i][1].append(Assets[i])

__BasicData = dict(_BasicData)
print(__BasicData)
BasicData = pd.DataFrame(__BasicData)
#=================================================


for i in range(len(Rates)):
    _RateData[i][1].append(Rates[i])
#========= 테스트용. (데이터프레임의 변수에는 각각 두개 이상의 요소들이 필요하다.)
for i in range(len(Rates)):
    _RateData[i][1].append(Rates[i])

__RateData = dict(_RateData)
print(__RateData)
RateData = pd.DataFrame(__RateData)
#=================================================


{'기업명': ['삼성전자', '삼성전자'], '회계연도': [35993876.0, 35993876.0], '매출액': [26407832.0, 26407832.0], '영업이익': [36345117.0, 36345117.0], '법인세차감전순이익': [236806988.0, 236806988.0], '당기순이익': [378235718.0, 378235718.0], '자산총계': [198215579.0, 198215579.0], '유동자산': [75604351.0, 75604351.0], '자본총계': [275948016.0, 275948016.0], '자본금': [897514.0, 897514.0], '부채총계': [102287702.0, 102287702.0], '비유동부채': [180020139.0, 180020139.0], '비유동자산': [26683351.0, 26683351.0], '이익잉여금': [271068211.0, 271068211.0]}
{'기업명': ['삼성전자', '삼성전자'], 'ROA': [0.0698184511490266, 0.0698184511490266], 'ROS': [0.1115162699506148, 0.1115162699506148], '매출총이익률': [0.1115162699506148, 0.1115162699506148], '매출액영업이익률': [0.15199668009797077, 0.15199668009797077], '매출액계속사업이익률': [0.1534799175774323, 0.1534799175774323], '총자산회전율': [0.6260830924487147, 0.6260830924487147], '유동자산회전율': [1.1946941264389719, 1.1946941264389719], '비유동자산회전율': [1.315447201160088, 1.315447201160088], '부채구성비율': [0.27043374576274154, 0.27043374576274154], '부채비율': [0.37067

In [24]:
BasicData

,기업명,회계연도,매출액,영업이익,법인세차감전순이익,당기순이익,자산총계,유동자산,자본총계,자본금,부채총계,비유동부채,비유동자산,이익잉여금
0,삼성전자,35993876.0,26407832.0,36345117.0,236806988.0,378235718.0,198215579.0,75604351.0,275948016.0,897514.0,102287702.0,180020139.0,26683351.0,271068211.0
1,삼성전자,35993876.0,26407832.0,36345117.0,236806988.0,378235718.0,198215579.0,75604351.0,275948016.0,897514.0,102287702.0,180020139.0,26683351.0,271068211.0


In [25]:
RateData

,기업명,ROA,ROS,매출총이익률,매출액영업이익률,매출액계속사업이익률,총자산회전율,유동자산회전율,비유동자산회전율,부채구성비율,부채비율
0,삼성전자,0.069818,0.111516,0.111516,0.151997,0.15348,0.626083,1.194694,1.315447,0.270434,0.370677
1,삼성전자,0.069818,0.111516,0.111516,0.151997,0.15348,0.626083,1.194694,1.315447,0.270434,0.370677
